# Basic usage of `DEMAP`

In this notebook, we will see how to install and use `DEMAP` to extract river
network from DEM.

## Install `DEMAP`

Download the source code from Github and install it using `pip` locally:

```Shell
git clone https://github.com/laijingtao/demap.git
pip install ./demap
```

In [ ]:
import demap

## Accelerate with `Numba`

Many methods in `DEMAP` can be accelerated using `Numba`. If `Numba` is already
installed, these methods will automatically detect it and run much faster.
[Here](https://numba.pydata.org/numba-doc/latest/user/installing.html) is the
guide for installing `Numba`.

## Process DEM
`demap.process_dem()` provides an "all-in-one" method to process the DEM file.

It fills the local depressions in DEM, calculate flow direction and drainage
area, and generate a stream network.

In [ ]:
demfile = 'olympics_500m_dem.tif'
res = demap.process_dem(demfile, drainage_area_threshold=1e5, base_level=0.5)

`demap.process_dem()` returns a python dictionary that contains the DEM data, 
stream network data, and some other relative information.

What is useful for us is the DEM and stream network.

In [ ]:
dem = res['dem']
stream_network = res['stream_network']

The river network is stored as a `demap.StreamNetwork` object.

We can save the stream network in shapefile to view it in GIS software:

In [ ]:
demap.network_to_shp(stream_network, 'demap_stream_network')

## Extract a stream (or river, channel, valley, etc.)
Given a channel head, we can extract a stream from the stream network:

In [ ]:
head_x, head_y = 454435, 5292993
example_stream = stream_network.extract_from_xy(head_x, head_y, direction='down')

`StreamNetwork.extract_from_xy()` returns a new `StreamNetwork` with only extracted sub-network.

This means `example_stream` is a `StreamNetwork` with only one stream, and we want to convert it to a `Stream` object.

In [ ]:
example_stream = example_stream.to_streams()[0]

and then plot its longitudinal profile:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(example_stream.dataset['distance_upstream'], example_stream.get_value(dem))

A stream (or river, channel, valley, etc.) is stored as a `demap.Stream` object.
`Stream.dataset` is a `xarray.Dataset` that contains the data associated with
the stream. For example, we use `distance_upstream` in the above code.

In addition, `demap.Stream` provides a group of useful methods. We use
`Stream.get_value()` to extract the elevation along the stream from the DEM
data.

Get the x, y coordinates of this stream:

In [ ]:
x, y = example_stream.xy()
print(x, y)

or latitude, longitude:

In [ ]:
lat, lon = example_stream.latlon()
print(lat, lon)

Extract a valley cross-sectional profile and plot:

In [ ]:
swath = demap.valley_xsec_at_xy(dem, example_stream, x=444689, y=5285903, length=5e3) # length of the cross-sectional profile
plt.plot(swath.dist, swath.z)

Extract a series of valley cross-sectional profiles:

In [ ]:
swath_list, anchor_point_list = demap.xsec_along_valley(dem, example_stream, length=5e3, spacing=5e3) # spacing controls the distance between two cross sections.

for swath in swath_list:
    plt.plot(swath.dist, swath.z)

## Extract the stream network in a catchment

Given an outlet, we can call `extract_from_xy(direction='up')` to extract
streams upstream, i.e., the stream network in a catchment.

In [ ]:
outlet_x, outlet_y = 403689, 5266903
example_network = stream_network.extract_from_xy(outlet_x, outlet_y, direction='up')

Convert this network into a list of streams:

In [ ]:
example_stream_list = example_network.to_streams()

and then plot the longitudinal profiles of all streams:

In [ ]:
for s in example_stream_list:
    plt.plot(s.dataset['distance_upstream'], s.get_value(dem))

## Merge two `StreamNetwork` objects

In [ ]:
sn1 = stream_network.extract_from_xy(452679.5,5294614.3, direction='down')
sn2 = stream_network.extract_from_xy(455160.9,5293049.7, direction='down')

In [ ]:
merged = demap.merge_stream_network(sn1, sn2)

In [ ]:
for s in merged.to_streams():
    plt.plot(s.dataset['distance_upstream'], s.get_value(dem))